Devin Winslow, Sidney Rood

IS 475 

Project Part 2

12/12/2023

Table Creation

In [17]:
CREATE TABLE pTask
(TaskID int PRIMARY KEY,
TaskDescription VARCHAR(25))

Commands completed successfully.

Total execution time: 00:00:00.011

In [19]:
CREATE TABLE pCustomer
(CustomerID INT PRIMARY KEY,
CustomerName VARCHAR(30) NOT NULL)

Commands completed successfully.

Total execution time: 00:00:00.008

In [1]:
CREATE TABLE pWorkOrder
(
WorkOrderID INT PRIMARY KEY,
DateProposed DATETIME NOT NULL,
DateAccepted DATETIME NOT NULL,
CustomerID INT FOREIGN KEY REFERENCES pCustomer(CustomerID) NOT NULL,
PrimaryWorkOrderID INT FOREIGN KEY REFERENCES pWorkOrder(WorkOrderID)
)

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'pWorkOrder' in the database.

Total execution time: 00:00:00.009

In [38]:
DROP TABLE pWorkOrder

Commands completed successfully.

Total execution time: 00:00:00.013

In [42]:
CREATE TABLE pTimeSheet
(TimeSheetID INT Identity(1,1) Primary Key,
WorkOrderID INT FOREIGN KEY REFERENCES pWorkOrder(WorkOrderID),
TaskID INT FOREIGN KEY REFERENCES pTask(TaskID),
HoursWorked Decimal(4,2),
HourlyPayRate DECIMAL(8,2))

Commands completed successfully.

Total execution time: 00:00:00.017

In [41]:
DROP TABLE pTimeSheet

Commands completed successfully.

Total execution time: 00:00:00.016

In [4]:
CREATE TABLE pWorkTask
(WorkOrderID INT FOREIGN KEY REFERENCES pWorkOrder(WorkOrderID) NOT NULL,
TaskId INT FOREIGN KEY REFERENCES pTask(TaskID) NOT NULL,
DateStarted DateTime,
DateCompleted DateTime,
EstHours INT,
EstLaborCost DECIMAL,
EstMaterialCost DECIMAL,
PRIMARY KEY (WorkOrderID, TaskID))

Commands completed successfully.

Total execution time: 00:00:00.016

In [48]:
CREATE TABLE pMaterials
(MaterialAssignedID INT PRIMARY KEY,
WorkOrderID INT FOREIGN KEY REFERENCES pWorkOrder(WorkOrderID) NOT NULL,
TaskId INT FOREIGN KEY REFERENCES pTask(TaskID) NOT NULL,
Quantity DECIMAL(8,2),
CostPerUOM DECIMAL(8,2))


Commands completed successfully.

Total execution time: 00:00:00.017

STORED PROCEDURES AND FUNCTIONS

Procedure 1 <span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">Take </span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">input (See Question 2)</span>

In [131]:
CREATE PROCEDURE SelectWorkOrder @WorkOrderID INT 
AS
SELECT DISTINCT wt.WorkOrderID,
wt.TaskID,
t.TaskDescription,
wt.EstHours,
ISNULL(tss.ActualHours,0) ActualHours,
wt.EstLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(tss.ActualLaborCost,0)) ActualLaborCost,
wt.EstMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ms.ActualMaterialCost,0)) ActualMaterialCost,
CASE WHEN DateCompleted IS NULL THEN Esthours-CONVERT(DECIMAL(8,2),ISNULL(tss.ActualLaborCost,0)) ELSE 0 END LaborToFinish,
CASE WHEN DateCompleted IS NULL THEN EstMaterialCost-CONVERT(DECIMAL(8,2),ISNULL(ms.ActualMaterialCost,0)) ELSE 0 END MaterialToFinish,
CASE WHEN DateStarted IS NULL THEN 'Not Started' WHEN DateCompleted IS NULL THEN 'In Progress' ELSE 'Completed' END StatusMessage
FROM pWorkTask wt 
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pTask t 
ON t.TaskID = wt.TaskId
LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE wt.WorkOrderID = @WorkOrderID
ORDER BY TaskID

Commands completed successfully.

Total execution time: 00:00:00.014

Procedure 2 <span style="font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre; color: rgb(0, 0, 255);">Take</span> <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">&nbsp;input</span>

In [40]:
CREATE PROCEDURE CalculateMinCostDiff
    @CostType VARCHAR(25) -- 'Labor', 'Material', 'Total'
AS
BEGIN
    WITH Summary AS (
    SELECT wt.WorkOrderID,
    SUM (EstHours) EstHours,
    SUM(tss.ActualHours) ActualHours,
    SUM(tss.ActualLaborCost) ActualLaborCost,
    SUM(wt.EstLaborCost) EstLaborCost,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) LaborCostDiff,
    SUM(wt.EstMaterialCost) EstMaterialCost,
    SUM(ms.ActualMaterialCost) ActualMaterialCost,
    SUM(wt.EstMaterialCost - ms.ActualMaterialCost) MatCostDiff,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) + SUM(wt.EstMaterialCost - ms.ActualMaterialCost) TotalCostDiff
    FROM pWorkTask wt 
    LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE DATECOMPLETED IS NOT NULL AND wt.WorkOrderID NOT IN (SELECT WorkOrderID FROM pWorkTask wt WHERE DateCompleted IS NULL)
GROUP BY wt.WorkOrderID
)

    SELECT 
        ss.WorkOrderID,
        c.CustomerName,
        dbo.YMDDash(MIN(DateStarted)) AS DateStarted,
        dbo.YMDDash(MAX(DateCompleted)) AS DateCompleted,
        ss.EstHours,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualHours, 0)) AS ActualHours,
        ss.EstLaborCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualLaborCost, 0)) AS ActualLaborCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.LaborCostDiff, 0)) AS LaborCostDiff,
        ss.EstMaterialCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualMaterialCost, 0)) AS ActualMaterialCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.MatCostDiff, 0)) AS MatCostDiff,
        ss.TotalCostDiff
    FROM Summary ss
    LEFT JOIN pWorkTask wt ON ss.WorkOrderID = wt.WorkOrderID
    LEFT JOIN pTimeSheet ts ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
    LEFT JOIN pMaterials m ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
    LEFT JOIN pWorkOrder wo ON wt.WorkOrderID = wo.WorkOrderID
    LEFT JOIN pCustomer c ON c.CustomerID = wo.CustomerID
    WHERE
        (@CostType = 'Labor' AND LaborCostDiff = (SELECT MIN(LaborCostDiff) FROM Summary))
        OR
        (@CostType = 'Material' AND MatCostDiff = (SELECT MIN(MatCostDiff) FROM Summary))
        OR
        (@CostType = 'Total' AND TotalCostDiff = (SELECT MIN(TotalCostDiff) FROM Summary))
    GROUP BY
        ss.WorkOrderID, ss.EstHours, ss.ActualHours, ss.ActualLaborCost, ss.EstLaborCost, ss.LaborCostDiff,
        ss.EstMaterialCost, ss.ActualMaterialCost, ss.MatCostDiff, c.CustomerName, ss.TotalCostDiff
    ORDER BY
        WorkOrderID;
END;

Commands completed successfully.

Total execution time: 00:00:00.010

Function 1 (See Question 2)

In [140]:
-- Rounding and Converting to Decimal Function
CREATE FUNCTION NullRound (
    @value DECIMAL
)
RETURNS DECIMAL(8,2)
BEGIN
RETURN CONVERT(DECIMAL(8,2),ISNULL(@value,0))
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.007

Function 2

In [30]:
-- Rounding and Converting to Decimal Function
CREATE FUNCTION YMDDash (
    @Date DATETIME
)
RETURNS VARCHAR(12)
BEGIN
RETURN CONCAT(YEAR(@Date),'-',MONTH(@Date),'-',DAY(@Date))
END 
GO

Commands completed successfully.

Total execution time: 00:00:00.008

In [29]:
DROP FUNCTION YMDDash

Commands completed successfully.

Total execution time: 00:00:00.006

Q1

In [18]:
SELECT ts.WorkOrderID,
ts.TaskID,
TaskDescription,
COUNT(TimeSheetID) CountofTimeSheets,
SUM(HoursWorked) TotalHoursWorked
FROM pTask t 
RIGHT JOIN pTimeSheet ts 
ON t.TaskID = ts.TaskID
GROUP BY WorkOrderID,ts.TaskID,TaskDescription
ORDER BY WorkOrderID,ts.TaskID

(44 rows affected)

Total execution time: 00:00:00.011

WorkOrderID,TaskID,TaskDescription,CountofTimeSheets,TotalHoursWorked
NULL,NULL,NULL,7,9.60
16885,110,Remove Existing Floor,2,7.00
16885,130,Install sub-floor,6,23.50
16885,140,Design mosaic/glass,4,9.00
16885,150,Prepare sub floor,1,4.50
16885,160,Install tile floor,5,19.25
16885,170,Grout,10,41.65
16885,270,General demolition,2,7.50
32687,110,Remove Existing Floor,2,7.50
32687,130,Install sub-floor,2,8.00


Q2

In [118]:
CREATE VIEW vTimeSheetSummary AS
WITH tsTimeSheet AS 
(SELECT WorkOrderID,
TaskID,
SUM(HoursWorked) HoursWorked,
SUM(HoursWorked) * HourlyPayRate LaborCosts
FROM pTimeSheet
GROUP BY WorkOrderID, TaskID, HourlyPayRate)

SELECT WorkOrderID,
TaskID,
SUM(HoursWorked) ActualHours,
SUM(LaborCosts) ActualLaborCost
FROM tsTimeSheet
GROUP BY WorkOrderID, TaskID

Commands completed successfully.

Total execution time: 00:00:00.007

In [122]:
CREATE VIEW vMaterialSummary AS
WITH mMaterials AS 
(SELECT WorkOrderID,
TaskID,
SUM(Quantity * CostPerUOM) MaterialCost
FROM pMaterials
GROUP BY WorkOrderID, TaskID, CostPerUOM)

SELECT msb.WorkOrderID,
msb.TaskID,
SUM(msb.MaterialCost) ActualMaterialCost
FROM mMaterials msb
GROUP BY msb.WorkOrderID,msb.TaskID

: Msg 2714, Level 16, State 3, Procedure vMaterialSummary, Line 1
There is already an object named 'vMaterialSummary' in the database.

Total execution time: 00:00:00.001

In [145]:
SELECT DISTINCT wt.WorkOrderID,
wt.TaskID,
t.TaskDescription,
wt.EstHours,
ISNULL(tss.ActualHours,0) ActualHours,
wt.EstLaborCost,
dbo.NullRound(tss.ActualLaborCost) ActualLaborCost,
wt.EstMaterialCost,
dbo.NullRound(ms.ActualMaterialCost) ActualMaterialCost,
CASE WHEN DateCompleted IS NULL THEN Esthours-dbo.NullRound(tss.ActualLaborCost) ELSE 0 END LaborToFinish,
CASE WHEN DateCompleted IS NULL THEN EstMaterialCost-dbo.NullRound(ms.ActualMaterialCost) ELSE 0 END MaterialToFinish,
CASE WHEN DateStarted IS NULL THEN 'Not Started' WHEN DateCompleted IS NULL THEN 'In Progress' ELSE 'Completed' END StatusMessage
FROM pWorkTask wt 
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pTask t 
ON t.TaskID = wt.TaskId
LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE wt.WorkOrderID = 16885
ORDER BY TaskID

(10 rows affected)

Total execution time: 00:00:00.030

WorkOrderID,TaskID,TaskDescription,EstHours,ActualHours,EstLaborCost,ActualLaborCost,EstMaterialCost,ActualMaterialCost,LaborToFinish,MaterialToFinish,StatusMessage
16885,110,Remove Existing Floor,6,7.00,75,77.00,25,23.00,0.00,0.00,Completed
16885,130,Install sub-floor,22,23.50,280,316.00,360,261.00,0.00,0.00,Completed
16885,140,Design mosaic/glass,10,9.00,240,221.00,10,18.00,0.00,0.00,Completed
16885,150,Prepare sub floor,5,4.50,61,66.00,250,243.00,0.00,0.00,Completed
16885,160,Install tile floor,18,19.25,230,234.00,1530,1795.00,0.00,0.00,Completed
16885,170,Grout,33,41.65,450,600.00,40,46.00,0.00,0.00,Completed
16885,180,Seal and finish work,17,0.00,180,0.00,25,20.00,17.00,5.00,In Progress
16885,240,Install tile wall,16,0.00,252,0.00,1625,1971.00,16.00,-346.00,In Progress
16885,260,Build mosaic/glass,16,0.00,384,0.00,110,0.00,16.00,110.00,Not Started
16885,270,General demolition,8,7.50,100,83.00,0,0.00,0.00,0.00,Completed


In [142]:
CREATE PROCEDURE SelectWorkOrder @WorkOrderID INT 
AS
SELECT DISTINCT wt.WorkOrderID,
wt.TaskID,
t.TaskDescription,
wt.EstHours,
ISNULL(tss.ActualHours,0) ActualHours,
wt.EstLaborCost,
dbo.NullRound(tss.ActualLaborCost) ActualLaborCost,
wt.EstMaterialCost,
dbo.NullRound(ms.ActualMaterialCost) ActualMaterialCost,
CASE WHEN DateCompleted IS NULL THEN Esthours-dbo.NullRound(tss.ActualLaborCost) ELSE 0 END LaborToFinish,
CASE WHEN DateCompleted IS NULL THEN EstMaterialCost-dbo.NullRound(ms.ActualMaterialCost) ELSE 0 END MaterialToFinish,
CASE WHEN DateStarted IS NULL THEN 'Not Started' WHEN DateCompleted IS NULL THEN 'In Progress' ELSE 'Completed' END StatusMessage
FROM pWorkTask wt 
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pTask t 
ON t.TaskID = wt.TaskId
LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE wt.WorkOrderID = @WorkOrderID
ORDER BY TaskID

: Msg 2714, Level 16, State 3, Procedure SelectWorkOrder, Line 1
There is already an object named 'SelectWorkOrder' in the database.

Total execution time: 00:00:00.002

In [139]:
EXEC SelectWorkOrder '16885'

(10 rows affected)

Total execution time: 00:00:00.023

WorkOrderID,TaskID,TaskDescription,EstHours,ActualHours,EstLaborCost,ActualLaborCost,EstMaterialCost,ActualMaterialCost,LaborToFinish,MaterialToFinish,StatusMessage
16885,110,Remove Existing Floor,6,7.00,75,77.00,25,22.83,0.00,0.00,Completed
16885,130,Install sub-floor,22,23.50,280,316.08,360,261.40,0.00,0.00,Completed
16885,140,Design mosaic/glass,10,9.00,240,220.50,10,18.00,0.00,0.00,Completed
16885,150,Prepare sub floor,5,4.50,61,66.38,250,242.98,0.00,0.00,Completed
16885,160,Install tile floor,18,19.25,230,233.89,1530,1794.74,0.00,0.00,Completed
16885,170,Grout,33,41.65,450,599.95,40,46.00,0.00,0.00,Completed
16885,180,Seal and finish work,17,0.00,180,0.00,25,19.95,17.00,5.05,In Progress
16885,240,Install tile wall,16,0.00,252,0.00,1625,1970.99,16.00,-345.99,In Progress
16885,260,Build mosaic/glass,16,0.00,384,0.00,110,0.00,16.00,110.00,Not Started
16885,270,General demolition,8,7.50,100,83.44,0,0.00,0.00,0.00,Completed


Q3

In [32]:
SELECT DISTINCT wt.WorkOrderID,
wt.TaskID,
t.TaskDescription,
dbo.YMDDash(DateCompleted) DateCompleted,
wt.EstHours,
CONVERT(DECIMAL(8,2),ISNULL(tss.ActualHours,0)) ActualHours,
wt.EstLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(tss.ActualLaborCost,0)) ActualLaborCost,
wt.EstLaborCost - CONVERT(DECIMAL(8,2),ISNULL(tss.ActualLaborCost,0)) LaborCostDiff,
wt.EstMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ms.ActualMaterialCost,0)) ActualMaterialCost,
wt.EstMaterialCost - CONVERT(DECIMAL(8,2),ISNULL(ms.ActualMaterialCost,0)) MatCostDiff
FROM pWorkTask wt 
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pTask t 
ON t.TaskID = wt.TaskId
LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE DATECOMPLETED IS NOT NULL AND wt.WorkOrderID NOT IN (SELECT WorkOrderID FROM pWorkTask wt WHERE DateCompleted IS NULL)
ORDER BY WorkOrderID, TaskID

(34 rows affected)

Total execution time: 00:00:00.045

WorkOrderID,TaskID,TaskDescription,DateCompleted,EstHours,ActualHours,EstLaborCost,ActualLaborCost,LaborCostDiff,EstMaterialCost,ActualMaterialCost,MatCostDiff
32687,110,Remove Existing Floor,2021-7-31,8,7.50,88,82.50,5.50,0,0.00,0.00
32687,130,Install sub-floor,2021-8-4,7,8.00,113,92.00,21.00,95,98.95,-3.95
32687,150,Prepare sub floor,2021-8-7,3,4.50,50,66.38,-16.38,110,105.00,5.00
32687,160,Install tile floor,2021-8-14,10,10.00,180,160.00,20.00,13550,13810.60,-260.60
32687,170,Grout,2021-8-18,6,6.50,90,95.88,-5.88,12,12.87,-0.87
32687,180,Seal and finish work,2021-8-22,4,4.50,60,65.25,-5.25,10,9.98,0.02
55841,130,Install sub-floor,2020-7-15,8,7.50,83,90.00,-7.00,83,94.90,-11.90
55841,150,Prepare sub floor,2020-7-16,4,4.00,45,52.00,-7.00,110,99.98,10.02
55841,160,Install tile floor,2020-7-19,11,10.50,170,162.75,7.25,240,235.15,4.85
55841,170,Grout,2020-7-22,10,10.50,140,139.75,0.25,9,11.88,-2.88


Q4

In [38]:
WITH Summary AS (
    SELECT wt.WorkOrderID,
    SUM (EstHours) EstHours,
    SUM(tss.ActualHours) ActualHours,
    SUM(tss.ActualLaborCost) ActualLaborCost,
    SUM(wt.EstLaborCost) EstLaborCost,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) LaborCostDiff,
    SUM(wt.EstMaterialCost) EstMaterialCost,
    SUM(ms.ActualMaterialCost) ActualMaterialCost,
    SUM(wt.EstMaterialCost - ms.ActualMaterialCost) MatCostDiff
    FROM pWorkTask wt 
    LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
GROUP BY wt.WorkOrderID
)

SELECT ss.WorkOrderID,
c.CustomerName,
dbo.YMDDash(MIN(DateStarted)) DateStarted,
dbo.YMDDash(Max(DateCompleted)) DateCompleted,
ss.EstHours,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualHours,0)) ActualHours,
ss.EstLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualLaborCost,0)) ActualLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.LaborCostDiff,0)) LaborCostDiff,
ss.EstMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualMaterialCost,0)) ActualMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.MatCostDiff,0)) MatCostDiff,
CONVERT(DECIMAL(8,2),ISNULL(ss.LaborCostDiff,0)) + CONVERT(DECIMAL(8,2),ISNULL(ss.MatCostDiff,0)) TotalCostDiff
FROM Summary ss
LEFT JOIN pWorkTask wt 
ON ss.WorkOrderID = wt.WorkOrderID
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pWorkOrder wo 
ON wt.WorkOrderID = wo.WorkOrderID
LEFT JOIN pCustomer c 
ON c.CustomerID = wo.CustomerID
WHERE DATECOMPLETED IS NOT NULL AND wt.WorkOrderID NOT IN (SELECT WorkOrderID FROM pWorkTask wt WHERE DateCompleted IS NULL)
GROUP BY ss.WorkOrderID, ss.EstHours, ss.ActualHours, ss.ActualLaborCost, ss.EstLaborCost, ss.LaborCostDiff, ss.EstMaterialCost, ss.ActualMaterialCost, ss.MatCostDiff, c.CustomerName
ORDER BY WorkOrderID

(7 rows affected)

Total execution time: 00:00:00.072

WorkOrderID,CustomerName,DateStarted,DateCompleted,EstHours,ActualHours,EstLaborCost,ActualLaborCost,LaborCostDiff,EstMaterialCost,ActualMaterialCost,MatCostDiff,TotalCostDiff
32687,Bread Bakery,2021-7-31,2021-8-22,38,41.00,581,562.00,19.00,13777,14037.40,-260.40,-241.40
55841,AO Reid Construction,2020-7-15,2020-7-29,38,36.50,496,490.50,5.50,455,453.66,1.35,6.85
55873,AO Reid Construction,2020-8-14,2020-9-2,53,42.30,523,692.65,-169.65,455,227.94,227.07,57.42
55878,AO Reid Construction,2020-9-12,2020-9-30,38,38.00,518,531.13,-13.13,455,216.53,238.48,225.35
62254,Dew Drop Inn Luxury Suites,2021-10-14,2021-11-3,158,153.25,2296,2250.91,45.09,2932,1130.20,1801.80,1846.89
62257,Dew Drop Inn Luxury Suites,2021-10-30,2021-11-18,179,158.25,2447,2343.53,103.48,3082,1130.20,1951.80,2055.28
91584,Adam's Rib Restaurant,2021-3-31,2021-4-21,145,151.00,2239,2210.38,28.63,12615,12833.23,-218.23,-189.60


Q5

In [39]:
WITH Summary AS (
    SELECT wt.WorkOrderID,
    SUM (EstHours) EstHours,
    SUM(tss.ActualHours) ActualHours,
    SUM(tss.ActualLaborCost) ActualLaborCost,
    SUM(wt.EstLaborCost) EstLaborCost,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) LaborCostDiff,
    SUM(wt.EstMaterialCost) EstMaterialCost,
    SUM(ms.ActualMaterialCost) ActualMaterialCost,
    SUM(wt.EstMaterialCost - ms.ActualMaterialCost) MatCostDiff,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) + SUM(wt.EstMaterialCost - ms.ActualMaterialCost) TotalCostDiff
    FROM pWorkTask wt 
    LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE DATECOMPLETED IS NOT NULL AND wt.WorkOrderID NOT IN (SELECT WorkOrderID FROM pWorkTask wt WHERE DateCompleted IS NULL)
GROUP BY wt.WorkOrderID
)

SELECT ss.WorkOrderID,
c.CustomerName,
dbo.YMDDash(MIN(DateStarted)) DateStarted,
dbo.YMDDash(Max(DateCompleted)) DateCompleted,
ss.EstHours,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualHours,0)) ActualHours,
ss.EstLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualLaborCost,0)) ActualLaborCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.LaborCostDiff,0)) LaborCostDiff,
ss.EstMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.ActualMaterialCost,0)) ActualMaterialCost,
CONVERT(DECIMAL(8,2),ISNULL(ss.MatCostDiff,0)) MatCostDiff,
ss.TotalCostDiff
FROM Summary ss
LEFT JOIN pWorkTask wt 
ON ss.WorkOrderID = wt.WorkOrderID
LEFT JOIN pTimeSheet ts 
ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
LEFT JOIN pMaterials m
ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
LEFT JOIN pWorkOrder wo 
ON wt.WorkOrderID = wo.WorkOrderID
LEFT JOIN pCustomer c 
ON c.CustomerID = wo.CustomerID
WHERE TotalCostDiff = (SELECT MIN(TotalCostDiff) FROM Summary)
GROUP BY ss.WorkOrderID, ss.EstHours, ss.ActualHours, ss.ActualLaborCost, ss.EstLaborCost, ss.LaborCostDiff, ss.EstMaterialCost, ss.ActualMaterialCost, ss.MatCostDiff, c.CustomerName, ss.TotalCostDiff
ORDER BY WorkOrderID

(1 row affected)

Total execution time: 00:00:00.045

WorkOrderID,CustomerName,DateStarted,DateCompleted,EstHours,ActualHours,EstLaborCost,ActualLaborCost,LaborCostDiff,EstMaterialCost,ActualMaterialCost,MatCostDiff,TotalCostDiff
32687,Bread Bakery,2021-7-31,2021-8-22,38,41.00,581,562.00,19.00,13777,14037.40,-260.40,-241.3950


In [1]:
CREATE PROCEDURE CalculateMinCostDiff
    @CostType VARCHAR(25) -- 'Labor', 'Material', 'Total'
AS
BEGIN
    WITH Summary AS (
    SELECT wt.WorkOrderID,
    SUM (EstHours) EstHours,
    SUM(tss.ActualHours) ActualHours,
    SUM(tss.ActualLaborCost) ActualLaborCost,
    SUM(wt.EstLaborCost) EstLaborCost,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) LaborCostDiff,
    SUM(wt.EstMaterialCost) EstMaterialCost,
    SUM(ms.ActualMaterialCost) ActualMaterialCost,
    SUM(wt.EstMaterialCost - ms.ActualMaterialCost) MatCostDiff,
    SUM(wt.EstLaborCost - tss.ActualLaborCost) + SUM(wt.EstMaterialCost - ms.ActualMaterialCost) TotalCostDiff
    FROM pWorkTask wt 
    LEFT JOIN vTimeSheetSummary tss
ON tss.WorkOrderID = wt.WorkOrderID AND tss.TaskId = wt.TaskId
LEFT JOIN vMaterialSummary ms 
ON ms.WorkOrderID = wt.WorkOrderID AND ms.TaskId = wt.TaskId
WHERE DATECOMPLETED IS NOT NULL AND wt.WorkOrderID NOT IN (SELECT WorkOrderID FROM pWorkTask wt WHERE DateCompleted IS NULL)
GROUP BY wt.WorkOrderID
)

    SELECT 
        ss.WorkOrderID,
        c.CustomerName,
        dbo.YMDDash(MIN(DateStarted)) AS DateStarted,
        dbo.YMDDash(MAX(DateCompleted)) AS DateCompleted,
        ss.EstHours,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualHours, 0)) AS ActualHours,
        ss.EstLaborCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualLaborCost, 0)) AS ActualLaborCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.LaborCostDiff, 0)) AS LaborCostDiff,
        ss.EstMaterialCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.ActualMaterialCost, 0)) AS ActualMaterialCost,
        CONVERT(DECIMAL(8,2), ISNULL(ss.MatCostDiff, 0)) AS MatCostDiff,
        ss.TotalCostDiff
    FROM Summary ss
    LEFT JOIN pWorkTask wt ON ss.WorkOrderID = wt.WorkOrderID
    LEFT JOIN pTimeSheet ts ON ts.WorkOrderID = wt.WorkOrderID AND ts.TaskID = wt.TaskId
    LEFT JOIN pMaterials m ON m.WorkOrderID = wt.WorkOrderID AND m.TaskId = wt.TaskId
    LEFT JOIN pWorkOrder wo ON wt.WorkOrderID = wo.WorkOrderID
    LEFT JOIN pCustomer c ON c.CustomerID = wo.CustomerID
    WHERE
        (@CostType = 'Labor' AND LaborCostDiff = (SELECT MIN(LaborCostDiff) FROM Summary))
        OR
        (@CostType = 'Material' AND MatCostDiff = (SELECT MIN(MatCostDiff) FROM Summary))
        OR
        (@CostType = 'Total' AND TotalCostDiff = (SELECT MIN(TotalCostDiff) FROM Summary))
    GROUP BY
        ss.WorkOrderID, ss.EstHours, ss.ActualHours, ss.ActualLaborCost, ss.EstLaborCost, ss.LaborCostDiff,
        ss.EstMaterialCost, ss.ActualMaterialCost, ss.MatCostDiff, c.CustomerName, ss.TotalCostDiff
    ORDER BY
        WorkOrderID;
END;


Commands completed successfully.

Total execution time: 00:00:00.015

In [4]:
EXEC CalculateMinCostDiff 'Total'

(1 row affected)

Total execution time: 00:00:00.023

WorkOrderID,CustomerName,DateStarted,DateCompleted,EstHours,ActualHours,EstLaborCost,ActualLaborCost,LaborCostDiff,EstMaterialCost,ActualMaterialCost,MatCostDiff,TotalCostDiff
32687,Bread Bakery,2021-7-31,2021-8-22,38,41.00,581,562.00,19.00,13777,14037.40,-260.40,-241.3950
